In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
print(tf.__version__)

# Fully Connected Network
![FCN](FCN.png)

## Input Node

### Placeholders

We, or our clients, cna later supply their own data when they need to execute the computation


```python
tf.placeholder(dypte, shape=None, name=None)
```

## Weights
### Variables

#### tf.Variable

```python
# <it's worse>
# create variable with tf.Variable
s = tf.Variable(2, name="scalar")
m = tf.Variable([[0, 1], [2, 3]], name="matrix")
W = tf.Variable(tf.zeros([784, 10]))
```
#### tf.get_variable

```python
# <it's better>
# create variable with tf.get_variable
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())
```

### tf.Variables

```python
__init__(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None,
    use_resource=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableAggregation.NONE
)

```

### Initialize

**tf.Variable** should get initial_value to making  weights matrix

#### Initialize method for Neural Network
- **LeCun Normal Initialization**
- LeCun Uniform Initialization
- Xavier Initialization (Glorot Initialization)
- He Initialization

#### Candidate of initial_value in tf.Variable
```python
tf.zeros(shape, dtype=tf.float32, name=None)
tf.zeros_like(tensor, dtype=None, name=None)
tf.ones(shape, dtype=tf.float32, name=None)
tf.ones_like(tensor, dtype=tf.None, name=None)
tf.fill(dims, value, name=None)
tf.constant(value, dtype=None, name=None, name=Const)
tf.linspace(start, stop, num, name=None)
tf.range(start, limit=None, delta=1, name=range)
tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)
tf.random_shuffle(value, seed=None, name=None)
tf.set_random_seed(seed)
tf.random_crop(value, size, seed=None, name=None)
tf.multinomial(logits, num_samples, seed=None, name=None, output_dtype=None)
tf.random.gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)
```

#### Candidate of LeCun Normal Initialization
```python
tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
```

#### What difference tf.random_normal vs tf.truncated_normal
![truncated normal distribution](truncated_normal_distribution.png)

### Op level random seed
**Each new session restarts the random state**
```python
c = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    print(sess.run(c)) # >> -5.97319
```

```python
c = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    
with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    
```

### Op level ssed: each op keeps its own seed

```python
c = tf.random_uniform([], -10, 10, seed=2)
d = tf.random_uniform([], -10, 10, seed=2)

with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    print(sess.run(d)) # >> 3.57493
```

In [ ]:
c = tf.Variable(initial_value=tf.truncated_normal([784, 200], seed=2) , dtype=tf.float32)
d = tf.Variable(initial_value=tf.truncated_normal([200, 10], seed=2) , dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([c]))
    print(sess.run([d]))
    
print("\nTwo Array Matrix is Same!!")

### Randomly Generated Constants

```python3
tf.set_random_seed(seed)
```

In [ ]:
tf.set_random_seed(2)
c = tf.Variable(initial_value=tf.truncated_normal([784, 200]) , dtype=tf.float32)
d = tf.Variable(initial_value=tf.truncated_normal([200, 10]) , dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([c]))
    print(sess.run([d]))
    
print("\nTwo Array Matrix is Different!!")

# Fully Connected Network Using tf.Variable

![sad...](sad.jpg)

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

tf.set_random_seed(2)

(x_train, y_train), (x_test , y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train/255).astype(np.float32).reshape(-1, 784)
x_test = (x_test/255).astype(np.float32).reshape(-1, 784)

val_range = int(len(x_test) * 0.8)

x_val = x_test[:val_range, :]
y_val = y_test[:val_range]

x_test = x_test[val_range:, :]
y_test = y_test[val_range:]

# Declare Input Node
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.int32)

# Declare parameter
layer1_weights = tf.Variable(initial_value=tf.truncated_normal([784, 200]) , dtype=tf.float32)
layer2_weights = tf.Variable(initial_value=tf.truncated_normal([200, 10]) , dtype=tf.float32)

# build graph
feature1 = tf.matmul(x, layer1_weights)
activation1 = tf.sigmoid(feature1)
score = tf.matmul(activation1, layer2_weights)

prediction = tf.argmax(input = score, axis=1)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=score)

opt = tf.train.GradientDescentOptimizer(learning_rate=0.001)
optimizer = opt.minimize(loss)

softmaxLoss = tf.summary.scalar(name='softmax_loss', tensor=loss)

train_writer = tf.summary.FileWriter('./graphs/train_mnist_dnn', graph=tf.get_default_graph())
val_writer = tf.summary.FileWriter('./graphs/val_mnist_dnn', graph=tf.get_default_graph())

# Hyper Parameter
epoch = 100
batch_size=512
total_step = int(x_train.shape[0] / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print("total_step : {}".format(total_step))

train_loss_hist = []
val_loss_hist = []

for i in range(epoch):
    avg_train_loss = 0   
    avg_val_loss = 0 
    
    for step in range(total_step):
            batch_indices = np.random.choice(range(x_train.shape[0]), size=batch_size, replace=False)

            batch_xs = x_train[batch_indices]
            batch_ys = y_train[batch_indices]
            
            val_indices = np.random.choice(range(x_val.shape[0]), size=batch_size, replace=False)
            
            val_xs = x_val[val_indices]
            val_ys = y_val[val_indices]

            _, train_loss = sess.run([optimizer, loss], feed_dict={x:batch_xs, y:batch_ys})
            train_loss_sum = sess.run(softmaxLoss, feed_dict={x:batch_xs, y:batch_ys})
            
            val_loss, val_loss_sum = sess.run(fetches = [loss, softmaxLoss], feed_dict={x:val_xs, y:val_ys})               
            avg_train_loss += train_loss / total_step
            avg_val_loss += val_loss / total_step
    
    train_loss_hist.append(avg_train_loss)
    val_loss_hist.append(val_loss)

    train_writer.add_summary(train_loss_sum, global_step = epoch)
    val_writer.add_summary(val_loss_sum, global_step = epoch)
    
    if i % 10 == 0:
        print('epoch : {:3}, train_loss : {:.2f}, val_loss : {:.2f}'.format(i, avg_train_loss, avg_val_loss))
        
train_writer.close()
val_writer.close()

print("train finished")


plt.plot(train_loss_hist, label="train")
plt.plot(val_loss_hist, label="validation")
plt.legend()
plt.show()


for i in range(x_test.shape[0]):
    output = sess.run(fetches = [prediction], feed_dict={x:x_test[i].reshape(-1, 784)})
    print("value : {}, label : {}".format(output, y_test[i]))

    

1.12.0
total_step : 7500
epoch :   0, train_loss : 1.05, val_loss : 0.78


KeyboardInterrupt: 